In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
data = pd.read_csv('/content/drive/My Drive/train.csv') 
data.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct


In [4]:
print('Train columns with null values:\n', data.isnull().sum())
print("-"*10)


data.describe(include = 'all')

Train columns with null values:
 tripid                         0
additional_fare              202
duration                     202
meter_waiting                202
meter_waiting_fare           202
meter_waiting_till_pickup    202
pickup_time                    0
drop_time                      0
pick_lat                       0
pick_lon                       0
drop_lat                       0
drop_lon                       0
fare                         137
label                          0
dtype: int64
----------


,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
count,1.717600e+04,16974.000000,16974.000000,16974.000000,16974.000000,16974.000000,17176,17176,17176.000000,17176.000000,17176.000000,17176.000000,17039.000000,17176
unique,NaN,NaN,NaN,NaN,NaN,NaN,15613,15665,NaN,NaN,NaN,NaN,NaN,2
top,NaN,NaN,NaN,NaN,NaN,NaN,11/13/2019 17:25,12/7/2019 12:21,NaN,NaN,NaN,NaN,NaN,correct
freq,NaN,NaN,NaN,NaN,NaN,NaN,4,4,NaN,NaN,NaN,NaN,NaN,15495
mean,2.014549e+08,13.719651,1702.858077,629.074231,32.057666,112.466832,NaN,NaN,6.965499,79.998389,6.968587,79.997534,312.655466,NaN
std,7.456830e+06,98.975052,8120.160818,8063.260669,465.483573,442.054812,NaN,NaN,0.210760,0.243384,0.378669,0.359648,605.300565,NaN
min,1.891236e+08,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,5.937600,79.819100,5.941860,45.307700,0.000000,NaN
25%,1.947619e+08,10.500000,548.000000,33.000000,0.000000,26.000000,NaN,NaN,6.865077,79.870400,6.865625,79.870100,125.515000,NaN
50%,2.013582e+08,10.500000,1009.500000,123.000000,0.116600,73.000000,NaN,NaN,6.914105,79.898600,6.916160,79.899100,211.180000,NaN
75%,2.083081e+08,10.500000,1803.000000,351.000000,14.341800,144.000000,NaN,NaN,7.043057,79.956600,7.047195,79.960225,366.465000,NaN


In [5]:
data.pickup_time.head()

0    11/1/2019 0:20
1    11/1/2019 0:56
2    11/1/2019 1:08
3    11/1/2019 2:27
4    11/1/2019 3:34
Name: pickup_time, dtype: object

In [6]:
data['pickup_time'] = pd.to_datetime(data['pickup_time'])
data['drop_time'] = pd.to_datetime(data['drop_time'])
data.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
0,189123628,10.5,834.0,56.0,0.0000,64.0,2019-11-01 00:20:00,2019-11-01 00:34:00,6.86252,79.8993,6.90330,79.8783,270.32,correct
1,189125358,10.5,791.0,47.0,0.0000,134.0,2019-11-01 00:56:00,2019-11-01 01:09:00,6.88589,79.8984,6.91373,79.8923,197.85,correct
2,189125719,10.5,1087.0,80.0,0.0000,61.0,2019-11-01 01:08:00,2019-11-01 01:26:00,6.90839,79.8651,6.93669,79.9146,301.64,correct
3,189127273,10.5,598.0,271.0,15.6638,68.0,2019-11-01 02:27:00,2019-11-01 02:37:00,6.92570,79.8895,6.92748,79.8971,82.30,correct
4,189128020,NaN,NaN,NaN,NaN,NaN,2019-11-01 03:34:00,2019-11-01 03:51:00,6.87441,79.8615,6.84478,79.9290,358.39,correct


In [7]:
data['fare_time'] = data['drop_time']-data['pickup_time']
data['fare_time'] = data['fare_time'].dt.total_seconds()
data = data.drop(['pickup_time', 'drop_time'], axis=1)
data.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,label,fare_time
0,189123628,10.5,834.0,56.0,0.0000,64.0,6.86252,79.8993,6.90330,79.8783,270.32,correct,840.0
1,189125358,10.5,791.0,47.0,0.0000,134.0,6.88589,79.8984,6.91373,79.8923,197.85,correct,780.0
2,189125719,10.5,1087.0,80.0,0.0000,61.0,6.90839,79.8651,6.93669,79.9146,301.64,correct,1080.0
3,189127273,10.5,598.0,271.0,15.6638,68.0,6.92570,79.8895,6.92748,79.8971,82.30,correct,600.0
4,189128020,NaN,NaN,NaN,NaN,NaN,6.87441,79.8615,6.84478,79.9290,358.39,correct,1020.0


In [0]:
from geopy.distance import geodesic
geodesic((data['pick_lat'][0], data['pick_lon'][0]), (data['drop_lat'][0], data['drop_lon'][0])).meters
for i, row in data.iterrows():
  data.at[i,'fare_distance'] = geodesic((data['pick_lat'][i], data['pick_lon'][i]), (data['drop_lat'][i], data['drop_lon'][i])).meters

In [9]:
data = data.drop(['pick_lat', 'pick_lon', 'drop_lat', 'drop_lon'], axis=1)
data.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,label,fare_time,fare_distance
0,189123628,10.5,834.0,56.0,0.0000,64.0,270.32,correct,840.0,5072.064513
1,189125358,10.5,791.0,47.0,0.0000,134.0,197.85,correct,780.0,3151.779729
2,189125719,10.5,1087.0,80.0,0.0000,61.0,301.64,correct,1080.0,6302.417567
3,189127273,10.5,598.0,271.0,15.6638,68.0,82.30,correct,600.0,862.654437
4,189128020,NaN,NaN,NaN,NaN,NaN,358.39,correct,1020.0,8148.520802


In [11]:
# data = data.dropna()
# data.head()
data['additional_fare'].fillna(data['additional_fare'].dropna().mean(), inplace=True)
data['duration'].fillna(data['duration'].dropna().mean(), inplace=True)
data['meter_waiting'].fillna(data['meter_waiting'].dropna().mean(), inplace=True)
data['meter_waiting_fare'].fillna(data['meter_waiting_fare'].dropna().mean(), inplace=True)
data['meter_waiting_till_pickup'].fillna(data['meter_waiting_till_pickup'].dropna().mean(), inplace=True)
data.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,label,fare_time,fare_distance
0,189123628,10.5,834.0,56.0,0.0000,64.0,270.32,correct,840.0,5072.064513
1,189125358,10.5,791.0,47.0,0.0000,134.0,197.85,correct,780.0,3151.779729
2,189125719,10.5,1087.0,80.0,0.0000,61.0,301.64,correct,1080.0,6302.417567
3,189127273,10.5,598.0,271.0,15.6638,68.0,82.30,correct,600.0,862.654437
5,189129552,10.5,3407.0,182.0,0.0000,112.0,1065.02,correct,3420.0,24078.954547


In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# data['label'] = le.fit_transform(data.label)
labels = {"label": {"correct": 1, "incorrect": 0}}
data = data.replace(labels)
y = data['label']
x = data.drop(['label', 'tripid'], axis=1)
x.head()
y.head()
data.head(10)

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,label,fare_time,fare_distance
0,189123628,10.5,834.0,56.0,0.0000,64.0,270.32,1,840.0,5072.064513
1,189125358,10.5,791.0,47.0,0.0000,134.0,197.85,1,780.0,3151.779729
2,189125719,10.5,1087.0,80.0,0.0000,61.0,301.64,1,1080.0,6302.417567
3,189127273,10.5,598.0,271.0,15.6638,68.0,82.30,1,600.0,862.654437
5,189129552,10.5,3407.0,182.0,0.0000,112.0,1065.02,1,3420.0,24078.954547
6,189132829,10.5,1246.0,487.0,0.0000,133.0,266.62,1,1200.0,4782.372696
7,189135103,10.5,1333.0,295.0,17.1985,212.0,318.05,1,1320.0,5328.739857
8,189139296,10.5,360.0,80.0,4.6640,3.0,100.32,1,360.0,1032.484483
9,189138671,10.5,1539.0,588.0,33.9864,43.0,257.89,1,1560.0,2933.781893
10,189135814,10.5,26.0,19.0,0.0000,73.0,694.37,0,0.0,14303.040775


In [0]:
y = y.values
x = x.values

In [0]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 0)

In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier


random_forest = RandomForestClassifier(n_estimators=1000)
random_forest.fit(X_train, y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, y_train)
acc_random_forest = round(random_forest.score(X_test, y_test) * 100, 2)
acc_random_forest

95.34

In [0]:
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
RFC = RandomForestClassifier()

kfold = StratifiedKFold(n_splits=10)
## Search grid for optimal parameters
rf_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}


gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsRFC.fit(X_train,y_train)

RFC_best = gsRFC.best_estimator_

# Best score
gsRFC.best_score_

Fitting 10 folds for each of 54 candidates, totalling 540 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  2.4min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed: 10.9min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed: 33.5min
[Parallel(n_jobs=4)]: Done 540 out of 540 | elapsed: 33.9min finished


0.9513039354115408

In [0]:
acc_random_forest = round(RFC_best.score(X_test, y_test) * 100, 2)
acc_random_forest

95.23

In [0]:
test_data = pd.read_csv('/content/drive/My Drive/test.csv')

In [17]:
test_data.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare
0,213284604,10.5,924,42,2.4486,148,2/1/2020 0:38,2/1/2020 0:53,6.83454,79.8750,6.77490,79.8840,289.27
1,213286352,10.5,4249,20,0.0000,91,2/1/2020 1:02,2/1/2020 2:13,6.91168,79.8723,6.55091,79.9706,1912.70
2,213293973,10.5,1552,255,2.6588,23,2/1/2020 5:02,2/1/2020 5:28,6.92145,79.8478,6.90539,79.8989,394.00
3,213294622,10.5,462,16,0.0000,198,2/1/2020 5:30,2/1/2020 5:38,6.77433,79.9416,6.80401,79.9407,154.32
4,213298687,10.5,814,392,12.3692,69,2/1/2020 7:00,2/1/2020 7:14,6.97968,79.9130,6.98875,79.8914,147.47


In [18]:
test_data['pickup_time'] = pd.to_datetime(test_data['pickup_time'])
test_data['drop_time'] = pd.to_datetime(test_data['drop_time'])
test_data['fare_time'] = test_data['drop_time']-test_data['pickup_time']
test_data['fare_time'] = test_data['fare_time'].dt.total_seconds()
test_data = test_data.drop(['pickup_time', 'drop_time'], axis=1)
geodesic((test_data['pick_lat'][0], test_data['pick_lon'][0]), (test_data['drop_lat'][0], test_data['drop_lon'][0])).meters
for i, row in test_data.iterrows():
  test_data.at[i,'fare_distance'] = geodesic((test_data['pick_lat'][i], test_data['pick_lon'][i]), (test_data['drop_lat'][i], test_data['drop_lon'][i])).meters
test_data = test_data.drop(['pick_lat', 'pick_lon', 'drop_lat', 'drop_lon'], axis=1)
test_data.head()


,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare,fare_time,fare_distance
0,213284604,10.5,924,42,2.4486,148,289.27,900.0,6670.189338
1,213286352,10.5,4249,20,0.0000,91,1912.70,4260.0,41351.053381
2,213293973,10.5,1552,255,2.6588,23,394.00,1560.0,5920.043673
3,213294622,10.5,462,16,0.0000,198,154.32,480.0,3283.812573
4,213298687,10.5,814,392,12.3692,69,147.47,840.0,2588.982212


In [0]:
x = test_data.drop(['tripid'], axis=1)


In [0]:
x_val = x.values

In [0]:
Y_pred = random_forest.predict(x_val)

In [0]:
submission = pd.DataFrame({
        "tripid": test_data["tripid"],
        "prediction": Y_pred
    })
submission.to_csv('/content/drive/My Drive/submission.csv', index=False)